# 📑 Tutorials for PyPOTS Forecasting Models

## 📀 Preparing the **PhysioNet-2012** dataset for this tutorial

In [1]:
from pypots.data.generating import gene_physionet2012

# Load the PhysioNet-2012 dataset
physionet2012_dataset = gene_physionet2012()

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())


2023-05-16 08:40:32 [INFO]: Loading the dataset physionet_2012 with TSDB (https://github.com/WenjieDu/Time_Series_Database)...
2023-05-16 08:40:32 [INFO]: Starting preprocessing physionet_2012...


Dataset physionet_2012 has already been downloaded. Processing directly...
Dataset physionet_2012 has already been cached. Loading from cache directly...
Loaded successfully!
dict_keys(['n_classes', 'n_steps', 'n_features', 'train_X', 'train_y', 'val_X', 'val_y', 'test_X', 'test_y', 'scaler', 'test_X_intact', 'test_X_indicating_mask', 'val_X_intact', 'val_X_indicating_mask'])


In [2]:
# Assemble the datasets for training, validating, and testing.

dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
}

dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "X_intact": physionet2012_dataset['val_X_intact'],
    "indicating_mask": physionet2012_dataset['val_X_indicating_mask'],
}

dataset_for_testing = {
    "X": physionet2012_dataset['test_X'][:, :36],  # we only take the first 36 steps for model input,
    # and let the model to forecast the left 12 steps
}


## 🚀 An exmaple of **BTTF** for forecasting

In [3]:
from pypots.forecasting import BTTF
import numpy as np
# initialize the model
bttf = BTTF(
    36,
    physionet2012_dataset["n_features"],
    pred_step=12,  # forcast the left half
    rank=10,
    time_lags=[1, 2, 3, 10, 10 + 1, 10 + 2, 20, 20 + 1, 20 + 2],
    burn_iter=5,
    gibbs_iter=5,
    multi_step=1,
)


2023-05-16 08:41:02 [INFO]: No given device, using default device: cpu


In [4]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
bttf.fit(train_set=dataset_for_training, val_set=dataset_for_validating)
# BTTF does not need to run func fits().


/workspaces/PyPOTS/pypots/forecasting/bttf/model.py:334: UserWarning: Please run func forecast(X) directly.
  warnings.warn("Please run func forecast(X) directly.")


In [5]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
bttf_forecasting_results = bttf.forecast(dataset_for_testing)


In [6]:
from pypots.utils.metrics import cal_mae

# calculate mean absolute error on the ground truth (artificially-missing values)
testing_mae = cal_mae(
    bttf_forecasting_results,
    np.nan_to_num(physionet2012_dataset['test_X'][:, 36:]),
    (~np.isnan(physionet2012_dataset['test_X'][:, 36:])).astype(int),
)
print("Testing mean absolute error: %.4f" % testing_mae)


Testing mean absolute error: 0.8869
